In [1]:
#### Basic Setting

import  load_dotenv
load_dotenv.load_dotenv("../../All_LLM_tutorial/.env")

True

Summary, Graph 같은 잡스러운 인덱스가 많은데 전부 프롬프트로 쪼개고 merge하는거 같은데 한국어가 동작할거라 믿을 수 없다  
결국은 VectorStoreIndex만 chk`

In [2]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

# Load documents and build index
documents = SimpleDirectoryReader(
    "./data"
).load_data()
index = VectorStoreIndex.from_documents(documents)

In [3]:
from llama_index.core import Document
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.extractors import TitleExtractor
from llama_index.core.ingestion import IngestionPipeline, IngestionCache

# create the pipeline with transformations
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=50, chunk_overlap=0),
        TitleExtractor(),
        OpenAIEmbedding(),
    ]
)

# run the pipeline
nodes = pipeline.run(documents=documents)

Metadata length (30) is close to chunk size (50). Resulting chunks are less than 50 tokens. Consider increasing the chunk size or decreasing the size of your metadata to avoid this.
Metadata length (24) is close to chunk size (50). Resulting chunks are less than 50 tokens. Consider increasing the chunk size or decreasing the size of your metadata to avoid this.
Metadata length (25) is close to chunk size (50). Resulting chunks are less than 50 tokens. Consider increasing the chunk size or decreasing the size of your metadata to avoid this.


100%|██████████| 3/3 [00:04<00:00,  1.55s/it]


In [4]:
[i.get_text() for i in nodes[:5]]

['What I Worked On\n\nFebruary 2021\n\nBefore college the two main things I worked',
 "on, outside of school, were writing and programming. I didn't write essays.",
 'I wrote what beginning writers were supposed to write then, and probably still are: short stories.',
 'My stories were awful.',
 'They had hardly any plot, just characters with strong feelings, which I imagined made them deep.\n\nThe']

In [5]:
[i.metadata["document_title"] for i in nodes][0]

'"Exploring Creativity Through Writing and Programming: Crafting Emotionally Rich Short Stories and Awful Tales"'

In [17]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
)
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb

# chroma_client = chromadb.EphemeralClient()        ### 휘발성
chroma_client = chromadb.PersistentClient(path="./chroma_db_example")
chroma_collection = chroma_client.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

documents = SimpleDirectoryReader(
    "./data",
    input_files=["./data/temp_data.txt"]
).load_data()
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, 
    transformations=[SentenceSplitter(chunk_size=30, chunk_overlap=10)],
    embed_model=OpenAIEmbedding()
)

## transformations에 IngestionPipeline을 넣던 SplitterParser를 넣던 동작은 전부 동일

Metadata length (7) is close to chunk size (30). Resulting chunks are less than 50 tokens. Consider increasing the chunk size or decreasing the size of your metadata to avoid this.


In [18]:
index.as_retriever().retrieve("오늘 날씨 어때?")

[NodeWithScore(node=TextNode(id_='069dedf1-3b4e-4a4c-86c9-ef782044f329', embedding=None, metadata={'file_path': 'data/temp_data.txt', 'file_name': 'temp_data.txt', 'file_type': 'text/plain', 'file_size': 2249, 'creation_date': '2025-07-08', 'last_modified_date': '2025-07-08'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='bb149176-4d97-4edd-9cdb-be602b821abb', node_type='4', metadata={'file_path': 'data/temp_data.txt', 'file_name': 'temp_data.txt', 'file_type': 'text/plain', 'file_size': 2249, 'creation_date': '2025-07-08', 'last_modified_date': '2025-07-08'}, hash='733bb2504e3bb003974636a5eea9247bbf716b2c8940ea161cbd6443bd4f71ee'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='e20ff645-aff0-4330

In [20]:
# get collection
chroma_client = chromadb.PersistentClient(path="./chroma_db_example")
chroma_collection = chroma_client.get_or_create_collection("quickstart")

# assign chroma as the vector_store to the context
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# load your index from stored vectors
index = VectorStoreIndex.from_vector_store(
    vector_store, storage_context=storage_context
)

# create a query engine
query_engine = index.as_retriever()
response = query_engine.retrieve("오늘 날씨 어때?")
print(response)

[NodeWithScore(node=TextNode(id_='069dedf1-3b4e-4a4c-86c9-ef782044f329', embedding=None, metadata={'file_path': 'data/temp_data.txt', 'file_name': 'temp_data.txt', 'file_type': 'text/plain', 'file_size': 2249, 'creation_date': '2025-07-08', 'last_modified_date': '2025-07-08'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='bb149176-4d97-4edd-9cdb-be602b821abb', node_type='4', metadata={'file_path': 'data/temp_data.txt', 'file_name': 'temp_data.txt', 'file_type': 'text/plain', 'file_size': 2249, 'creation_date': '2025-07-08', 'last_modified_date': '2025-07-08'}, hash='733bb2504e3bb003974636a5eea9247bbf716b2c8940ea161cbd6443bd4f71ee'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='e20ff645-aff0-4330

### Update

In [26]:
doc_to_update = chroma_collection.get(limit=1)
doc_to_update["metadatas"][0] = {
    **doc_to_update["metadatas"][0],
    **{"author": "Paul Graham"},
}

In [27]:
chroma_collection.update(
    ids=[doc_to_update["ids"][0]], metadatas=[doc_to_update["metadatas"][0]]
)

In [28]:
updated_doc = chroma_collection.get(limit=1)
print(updated_doc["metadatas"][0])

{'creation_date': '2025-07-08', '_node_type': 'TextNode', 'file_name': 'temp_data.txt', 'file_path': 'data/temp_data.txt', 'doc_id': '6b331ad6-231f-4c2b-affa-96a654514218', 'file_type': 'text/plain', 'last_modified_date': '2025-07-08', '_node_content': '{"id_": "7034935c-8ffe-43fa-b500-73facc33e6fa", "embedding": null, "metadata": {"file_path": "data/temp_data.txt", "file_name": "temp_data.txt", "file_type": "text/plain", "file_size": 2249, "creation_date": "2025-07-08", "last_modified_date": "2025-07-08"}, "excluded_embed_metadata_keys": ["file_name", "file_type", "file_size", "creation_date", "last_modified_date", "last_accessed_date"], "excluded_llm_metadata_keys": ["file_name", "file_type", "file_size", "creation_date", "last_modified_date", "last_accessed_date"], "relationships": {"1": {"node_id": "6b331ad6-231f-4c2b-affa-96a654514218", "node_type": "4", "metadata": {"file_path": "data/temp_data.txt", "file_name": "temp_data.txt", "file_type": "text/plain", "file_size": 2249, "cre

### Metadata insertion

In [9]:
from llama_index.core.extractors import BaseExtractor


class CustomExtractor(BaseExtractor):
    async def aextract(self, nodes):
        metadata_list = [
            {
                "custom": i
            }
            for i, node in enumerate(nodes)
        ]
        return metadata_list

In [10]:
transformations = [
    SentenceSplitter(),
    CustomExtractor(),
    OpenAIEmbedding(),

]
pipeline = IngestionPipeline(
    transformations=transformations
)
nodes = pipeline.run(documents=documents)

In [11]:
[i.metadata for i in nodes[:3]]

[{'file_path': 'data/temp_data.txt',
  'file_name': 'temp_data.txt',
  'file_type': 'text/plain',
  'file_size': 2249,
  'creation_date': '2025-07-08',
  'last_modified_date': '2025-07-08',
  'custom': 0}]

## General pattern  
IngestionPipeline은 전처리 파이프라인 느낌, 따라서 index가 아닌 node를 반환(node, index는 저장 여부의 차이임)  
VectorStore는 여기서 저장도 붙여 index 생성 가능

In [12]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter

documents = SimpleDirectoryReader("./data").load_data()

# global
from llama_index.core import Settings

Settings.text_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=20)

# per-index
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[SentenceSplitter(chunk_size=1024, chunk_overlap=20)],
)

In [13]:
from llama_index.core import Document
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.extractors import TitleExtractor
from llama_index.core.ingestion import IngestionPipeline, IngestionCache

# create the pipeline with transformations
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(chunk_size=25, chunk_overlap=0),
        # TitleExtractor(),
        OpenAIEmbedding(),
    ]
)

# run the pipeline
nodes = pipeline.run(documents=[Document.example()])

Metadata length (9) is close to chunk size (25). Resulting chunks are less than 50 tokens. Consider increasing the chunk size or decreasing the size of your metadata to avoid this.
